**Note: This is my EDA.  It's not pretty or nice and probably has some mistakes along the way.  Check out the associated `.py` file for a linted/formatted version of the solution.**

In [175]:
import datetime
from pathlib import Path
import sys
import math

import polars as pl

# Gross hack to get jupyter to find hod stuff.
sys.path.append(str(Path(Path.cwd().parent)))
from hod.utils import get_data

# The Early Bird

(JAMES: Emphasis below mine when I think something will be useful.)

REF: https://hanukkah.bluebird.sh/5784/4/

The investigator called the phone number you found and left a message, and a man soon called back:

“Wow, that was years ago! It was quite an elegant tapestry.

“It took a lot of patience, but I did manage to get the dirt out of one section, which uncovered a superb owl. I put it up on my wall, and sometimes at night I swear I could hear the owl hooting.

“A few weeks later my bike chain broke on the way home, and I needed to get it fixed before work the next day. Thankfully, this woman I met on Tinder came over at 5am with her bike chain repair kit and some pastries from Noah’s. Apparently she liked to get up before dawn and claim the **first pastries that came out of the oven**.

“I didn’t have any money or I would’ve paid her for her trouble. She really liked the tapestry, though, so I wound up giving it to her.

“I don’t remember her name or anything else about her.”

Can you find the bicycle fixer’s phone number?

In [36]:
# > 2017


In [144]:
data = get_data(data_path=Path('./../data/5784/'))
df_customers = data["customers"]
df_orders = data["orders"]
df_orders_items = data["orders_items"]
df_products = data["products"]

df_pastries = (
    df_products
    .select(
        pl.col(["sku", "desc"])
    )
    .filter((
        pl.col("sku").str.contains("BKY"),
        ~pl.col("desc").str.contains("Bagel")
    ))
)

df_orders_containing_pastries = df_orders_items.join(df_pastries, on="sku", how="inner").select(pl.col("orderid")).unique()

df_pastry_order_data = df_orders.join(df_orders_containing_pastries, on="orderid", how="inner").select([pl.col(["orderid", "customerid", "ordered"])])

df_early_morning_pastry_orderers = df_pastry_order_data.filter((
    (pl.col("ordered").dt.hour() >= 0) &
    (pl.col("ordered").dt.hour() < 7)
))

# Lucky guess!

# customerid = df_early_morning_pastry_orderers.group_by(by="customerid").agg(pl.col("orderid").count()).sort(by="orderid", descending=True).item(row=0, column="customerid")

# df_customers.filter(pl.col("customerid") == customerid).item(row=0, column="phone")

In [145]:
MALE_NAMES = [
    "Adam", "Alvin", "Anthony", "Austin", "Bradley", "Brandon", "Brian", "Bryan", "Caleb", "Carl", "Charles", "Christopher", "Clifford", "Cody" "Cristian", "Daniel", "David", "George", "Glenn", "Gregory", "James", "Jared", "Jeremiah", "Jeremy", "John", "Jon", "Jonathan", "Jose", "Joshua", "Kevin", "Kurt", "Lawrence", "Marcus", "Mark", "Matthew", "Michael", "Mike", "Nick", "Norman", "Peter", "Richard", "Robert", "Ronald", "Ruben", "Samuel", "Scott", "Stanley", "Stephen", "Steve", "Timothy", "Todd", "Travis", "Walter", "Xavier",
]

def not_male_name(s: str) -> bool:
    for n in MALE_NAMES:
        if n in s:
            return False
    return True


df_suspects = (
    df_early_morning_pastry_orderers
    .join(df_customers, on="customerid")
    .select(pl.col(["orderid", "customerid", "ordered", "name"]))
    # Filter out male names.
    .filter((
        pl.col("name").map_elements(lambda s: not_male_name(s))
    ))
)

multiple_order_suspects = df_suspects.group_by(pl.col("customerid")).agg(pl.col("orderid").count().name.suffix("_count")).filter(pl.col("orderid_count") > 1).sort("orderid_count", descending=True).get_column("customerid")

df_multiple_order_suspects = df_suspects.filter(pl.col("customerid").is_in(multiple_order_suspects))

df_multiple_pastry_order_suspects = (df_multiple_order_suspects.join(df_orders_items, on="orderid")
     .join(df_products, on="sku")
     .select((pl.col("orderid", "customerid", "ordered", "name", "qty", "desc", "sku")))
).filter(pl.col("sku").str.contains("BKY")).sort(by="ordered")


In [192]:
df_sus_shopping_list_more_than_one_cookie = (
    df_multiple_pastry_order_suspects
    .group_by(["customerid", "orderid", "name", "ordered"])
    .agg([pl.col("desc").alias("shopping_list"), pl.col("qty").sum().alias("total_qty")])
    .sort("customerid", descending=True).filter(pl.col("total_qty") > 1)
    .select(
    (pl.col(["customerid", "ordered", "name"]), pl.col("ordered").dt.date().alias("date")))
)

def get_age(d: datetime.date) -> float:
    return (datetime.date.today() - d).total_seconds() / (60 * 60 * 24 * 365.25)

df_suspects_for_real = (
    df_sus_shopping_list_more_than_one_cookie.join(df_customers, on="customerid").sort("date")
    .select((pl.col(["name", "birthdate", "phone"]), pl.col("birthdate").map_elements(lambda d: get_age(d)).round(decimals=0).cast(pl.Int8).alias("age")))
    .unique()
    .sort("birthdate")
)


# REF: https://www.zippia.com/advice/tinder-statistics/
(    df_suspects_for_real
     # There's only a ~4% chance the person on Tinder is > 45.
    .filter(pl.col("age") <= 45)
    # There's only a 13% chance the person on Tinder is 35 - 44.
    # .filter(pl.col("age") < 35)
).join(df_customers, on="name")



name,birthdate,phone,age,customerid,address,citystatezip,birthdate_right,phone_right,timezone,lat,long
str,date,str,i8,i64,str,str,date,str,str,f64,f64
"""Renee Harmon""",1999-01-14,"""607-231-3605""",25,2749,"""7A Nassau Ave""","""Brooklyn, NY 1…",1999-01-14,"""607-231-3605""","""America/New_Yo…",40.72347,-73.95097
"""Melanie Owens""",1988-04-16,"""315-977-3650""",36,5999,"""170 W 48th St""","""Manhattan, NY …",1988-04-16,"""315-977-3650""","""America/New_Yo…",40.76246,-73.9876


In [190]:
df_customers.filter(pl.col("phone") == "917-288-9635")

customerid,name,address,citystatezip,birthdate,phone,timezone,lat,long
i64,str,str,str,date,str,str,f64,f64
2550,"""Robert Morton""","""145-51 107th A…","""Jamaica, NY 11…",1999-07-08,"""917-288-9635""","""America/New_Yo…",40.68959,-73.80487
